In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import numpy as np
import re

# Constants
DATA_DIR = "software_monetization_dataset"  # Change this to your dataset folder
OUTPUT_DIR = "eda_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Sanitize filenames
def safe_filename(title):
    return re.sub(r'\W+', '_', title.lower())

# Load all CSV datasets from a specific folder
def load_datasets():
    datasets = {}
    for file in os.listdir(DATA_DIR):
        if file.endswith(".csv"):
            full_path = os.path.join(DATA_DIR, file)
            try:
                df = pd.read_csv(full_path)
                datasets[file] = df
                print(f"\n📁 Loaded '{file}' with shape {df.shape}")
                print(df.head(2))
            except Exception as e:
                print(f"❌ Failed to load {file}: {e}")
    return datasets

# Display dataset stats
def print_plot_info(title, data):
    print(f"\n📊 Plot: {title}")
    print(data.describe(include='all'))

# Plot distributions
def analyze_distributions(datasets):
    for name, df in datasets.items():
        for col in df.select_dtypes(include=np.number).columns:
            title = f"{name} - Distribution of {col}"
            safe_title = safe_filename(title)
            plt.figure()
            sns.histplot(df[col].dropna(), kde=True)
            plt.title(title)
            plt.savefig(f"{OUTPUT_DIR}/{safe_title}.png")
            plt.close()
            print_plot_info(title, df[[col]])

# Correlation heatmap
def correlation_analysis(datasets):
    for name, df in datasets.items():
        num_df = df.select_dtypes(include=np.number)
        if num_df.shape[1] >= 2:
            corr = num_df.corr()
            title = f"{name} - Correlation Matrix"
            safe_title = safe_filename(title)
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr, annot=True, cmap="coolwarm")
            plt.title(title)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/{safe_title}.png")
            plt.close()
            print_plot_info(title, corr)

# Outlier detection using z-score
def detect_outliers(datasets):
    for name, df in datasets.items():
        num_df = df.select_dtypes(include=np.number)
        for col in num_df.columns:
            col_data = df[[col]].dropna()
            if col_data.shape[0] > 10:
                z_scores = zscore(col_data)
                outliers = col_data[(np.abs(z_scores) > 3).any(axis=1)]
                title = f"{name} - Outliers in {col}"
                print_plot_info(title, outliers)
                if not outliers.empty:
                    plt.figure()
                    sns.boxplot(x=df[col])
                    plt.title(title)
                    safe_title = safe_filename(title)
                    plt.savefig(f"{OUTPUT_DIR}/{safe_title}.png")
                    plt.close()

# Customer segmentation with KMeans
def customer_segmentation(datasets):
    for name, df in datasets.items():
        if 'customer_id' in df.columns:
            num_df = df.select_dtypes(include=np.number).dropna()
            if num_df.shape[1] >= 2:
                scaler = StandardScaler()
                scaled = scaler.fit_transform(num_df)
                kmeans = KMeans(n_clusters=4, random_state=42)
                labels = kmeans.fit_predict(scaled)
                df['cluster'] = labels
                title = f"{name} - Customer Segmentation"
                fig = px.scatter_matrix(df, dimensions=num_df.columns[:3], color='cluster')
                fig.write_html(f"{OUTPUT_DIR}/{safe_filename(title)}.html")
                print(f"\n📊 {title}")
                print(df[['customer_id', 'cluster'] + list(num_df.columns[:3])].head())

# Run all EDA steps
def main_eda():
    datasets = load_datasets()
    analyze_distributions(datasets)
    correlation_analysis(datasets)
    detect_outliers(datasets)
    customer_segmentation(datasets)

main_eda()



📁 Loaded 'customers.csv' with shape (500, 15)
  Customer_ID                  Company_Name Industry_Type Company_Size  \
0      C00001                     Smith PLC    Government        large   
1      C00002  Newman, Johnson and Phillips     Education       medium   

   Employee_Count  Annual_Revenue      Location      State   Country  \
0            5902       417326886   Taylorshire  Minnesota  Djibouti   
1             508        18918315  North Daniel  Louisiana   Iceland   

   Years_in_Business   Segment  IT_Budget    Decision_Maker  \
0                 27  Standard    3765256    Jesse Anderson   
1                 34   Premium    3619060  Nichole Campbell   

               Contact_Email Registration_Date  
0        enewman@example.org        2023-04-18  
1  parkercameron@example.com        2024-06-14  

📁 Loaded 'customer_summary.csv' with shape (500, 9)
  Customer_ID  Total_Purchased  Total_Activated  Total_Contract_Value  \
0      C00001              337              258   

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.ensemble import IsolationForest
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import numpy as np
import re

# Constants
DATA_DIR = "software_monetization_dataset"  # change this if needed
OUTPUT_DIR = "eda_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def safe_filename(title):
    return re.sub(r'\W+', '_', title.lower())

def load_datasets():
    datasets = {}
    for file in os.listdir(DATA_DIR):
        if file.endswith(".csv"):
            path = os.path.join(DATA_DIR, file)
            try:
                df = pd.read_csv(path)
                datasets[file] = df
                print(f"\n📁 Loaded '{file}' with shape {df.shape}")
            except Exception as e:
                print(f"❌ Error loading {file}: {e}")
    return datasets

# Histogram / KDE
def analyze_distributions(datasets):
    for name, df in datasets.items():
        for col in df.select_dtypes(include=np.number).columns:
            clean_col = df[col].dropna()
            title = f"{name} - Histogram of {col}"
            counts, bins = np.histogram(clean_col, bins='auto')
            bin_data = pd.DataFrame({'bin_start': bins[:-1], 'bin_end': bins[1:], 'count': counts})
            print(f"\n📊 {title}")
            print(bin_data.to_string(index=False))
            bin_data.to_csv(f"{OUTPUT_DIR}/{safe_filename(title)}.csv", index=False)
            
            plt.figure()
            sns.histplot(clean_col, kde=True)
            plt.title(title)
            plt.savefig(f"{OUTPUT_DIR}/{safe_filename(title)}.png")
            plt.close()

# Correlation
def correlation_analysis(datasets):
    for name, df in datasets.items():
        num_df = df.select_dtypes(include=np.number)
        if num_df.shape[1] >= 2:
            corr = num_df.corr()
            title = f"{name} - Correlation Matrix"
            print(f"\n📊 {title}")
            print(corr.round(3).to_string())
            corr.to_csv(f"{OUTPUT_DIR}/{safe_filename(title)}.csv")
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(corr, annot=True, cmap="coolwarm")
            plt.title(title)
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_DIR}/{safe_filename(title)}.png")
            plt.close()

def detect_outliers(datasets):
    for name, df in datasets.items():
        for col in df.select_dtypes(include=np.number).columns:
            clean_col = df[col].dropna()
            if len(clean_col) < 10:
                continue
            z_scores = zscore(clean_col)
            outliers = clean_col[(np.abs(z_scores) > 3)]
            title = f"{name} - Outliers in {col}"
            print(f"\n📊 {title}")
            if outliers.empty:
                print("No significant outliers found.")
            else:
                outlier_data = pd.DataFrame({'value': outliers.values, 'z_score': z_scores[outliers.index]})
                print(outlier_data.to_string(index=False))
                outlier_data.to_csv(f"{OUTPUT_DIR}/{safe_filename(title)}.csv", index=False)
            
            plt.figure()
            sns.boxplot(x=clean_col)
            plt.title(title)
            plt.savefig(f"{OUTPUT_DIR}/{safe_filename(title)}.png")
            plt.close()

def customer_segmentation(datasets):
    for name, df in datasets.items():
        if 'customer_id' not in df.columns:
            continue
        num_df = df.select_dtypes(include=np.number).dropna()
        if num_df.shape[1] < 2:
            continue
        scaler = StandardScaler()
        scaled = scaler.fit_transform(num_df)
        kmeans = KMeans(n_clusters=4, random_state=42)
        labels = kmeans.fit_predict(scaled)
        df['cluster'] = labels
        title = f"{name} - Customer Clusters"

        cluster_df = df[['customer_id', 'cluster'] + list(num_df.columns)]
        print(f"\n📊 {title} (first 10 rows)")
        print(cluster_df.head(10).to_string(index=False))
        cluster_df.to_csv(f"{OUTPUT_DIR}/{safe_filename(title)}.csv", index=False)

        fig = px.scatter_matrix(df, dimensions=num_df.columns[:3], color='cluster')
        fig.write_html(f"{OUTPUT_DIR}/{safe_filename(title)}.html")

def main_eda():
    datasets = load_datasets()
    analyze_distributions(datasets)
    correlation_analysis(datasets)
    detect_outliers(datasets)
    customer_segmentation(datasets)

main_eda()



📁 Loaded 'customers.csv' with shape (500, 15)

📁 Loaded 'customer_summary.csv' with shape (500, 9)

📁 Loaded 'licenses.csv' with shape (3920, 44)

📁 Loaded 'products.csv' with shape (274, 15)

📁 Loaded 'product_performance.csv' with shape (274, 8)

📁 Loaded 'renewal_history.csv' with shape (5873, 16)

📁 Loaded 'usage_history.csv' with shape (76441, 16)

📁 Loaded 'vendors.csv' with shape (50, 18)

📁 Loaded 'vendor_performance.csv' with shape (50, 7)

📊 customers.csv - Histogram of Employee_Count
  bin_start     bin_end  count
  11.000000  236.181818    261
 236.181818  461.363636     40
 461.363636  686.545455     39
 686.545455  911.727273     23
 911.727273 1136.909091     24
1136.909091 1362.090909      6
1362.090909 1587.272727      3
1587.272727 1812.454545      2
1812.454545 2037.636364      4
2037.636364 2262.818182      3
2262.818182 2488.000000      0
2488.000000 2713.181818      1
2713.181818 2938.363636      3
2938.363636 3163.545455      6
3163.545455 3388.727273      2
338

IndexError: index 3882 is out of bounds for axis 0 with size 3850